In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Column / Row options
pd.options.display.max_columns = None
pd.options.display.max_rows = 20

# Two decimal places and the thousands separator
pd.options.display.float_format = '{:,.2f}'.format

# Import files

In [4]:
# Load the data frames
df_scores = pd.read_csv('/content/drive/MyDrive/Colab Data/KY EPSB/All Years/Unpublished/df_scores.csv')
ky_spending_df = pd.read_csv('/content/drive/MyDrive/Colab Data/KY EPSB/All Years/Unpublished/preprocessed_df.csv').loc[:,['End Year Code', 'School Code', 'Level Code', 'Reported Spending per student', 'Money Difference per school',]]

# Merge into one df
df = pd.merge(df_scores, ky_spending_df, on=['End Year Code', 'School Code', 'Level Code'], how='left')

# Sort df & Reset index
df.sort_values(by=['End Year', 'District Code', 'School Code'], inplace=True)
df.reset_index(drop=True, inplace=True)

# This predict_df will hold the predicted values.
predict_df = df.copy()


# Predict Classification: a NN Model

## Shuffle, Balance, Split, Scale

In [5]:
unscaled_df = predict_df.loc[:,['End Year Code', 'District Code', 'Level Code',
                                'Reported Spending per student', 'Money Difference per school',
                                'Proficiency Score', 'Classification Code', ]]

unscaled_df.dropna(axis =0, inplace=True)
unscaled_df.reset_index(drop=True, inplace=True)

#Shuffle df
unscaled_df = unscaled_df.sample(frac=1, random_state=15)

In [6]:
#Separate into input and targets
unscaled_inputs = unscaled_df.values[:,:-1]
targets = unscaled_df.values[:,-1]

# Split into training, validation, and test sets
unscaled_X_train, unscaled_X_set, y_train, y_set = train_test_split(unscaled_inputs, targets,
                                                                    test_size=0.2, random_state=15,
                                                                    stratify = targets)

unscaled_X_valid, unscaled_X_test, y_valid, y_test = train_test_split(unscaled_X_set, y_set,
                                                                      test_size=0.5, random_state=13,
                                                                      stratify = y_set)

In [7]:
# Scale data
epsb_scaler = StandardScaler()

# Calculate and store the mean and sd
epsb_scaler.fit(unscaled_X_train)

# Apply the scaler
scaled_X_train = epsb_scaler.transform(unscaled_X_train)
scaled_X_valid = epsb_scaler.transform(unscaled_X_valid)
scaled_X_test = epsb_scaler.transform(unscaled_X_test)

## Model

In [8]:
# Create the model
# Ran different rounds of the below model.  Some of the options I put in comments to the right.

input_size = 6
output_size = 3
hidden_layer_size = 15 # for this problem, optimal results were between 10-20 layers

model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layer_size, activation='tanh'),  # attempted to make this linear, and it only came to 77% valid accuracy or less
    tf.keras.layers.Dense(hidden_layer_size, activation='tanh'),
    tf.keras.layers.Dense(output_size, activation='softmax')
    ])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [9]:
# Fit the model

batch_size = 50  # started with batch size 25
max_epochs = 100 # started with 20 epochs

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

#early_stopping = tf.keras.callbacks.EarlyStopping(patience=5)

model.fit(scaled_X_train,
          y_train,
          batch_size=batch_size,
          epochs = max_epochs,
          callbacks = [early_stopping],
          validation_data = (scaled_X_valid, y_valid),
          verbose =2)

# Ran rounds using a multi-index of School Code & End Year.  But didn't get above a 78% validation accuracy

Epoch 1/100
100/100 - 5s - loss: 0.9829 - accuracy: 0.5370 - val_loss: 0.7677 - val_accuracy: 0.6984 - 5s/epoch - 49ms/step
Epoch 2/100
100/100 - 1s - loss: 0.6626 - accuracy: 0.7108 - val_loss: 0.5637 - val_accuracy: 0.7403 - 781ms/epoch - 8ms/step
Epoch 3/100
100/100 - 1s - loss: 0.5209 - accuracy: 0.7588 - val_loss: 0.4750 - val_accuracy: 0.7790 - 966ms/epoch - 10ms/step
Epoch 4/100
100/100 - 1s - loss: 0.4519 - accuracy: 0.7923 - val_loss: 0.4281 - val_accuracy: 0.8290 - 699ms/epoch - 7ms/step
Epoch 5/100
100/100 - 0s - loss: 0.4141 - accuracy: 0.8155 - val_loss: 0.4024 - val_accuracy: 0.8371 - 414ms/epoch - 4ms/step
Epoch 6/100
100/100 - 1s - loss: 0.3917 - accuracy: 0.8260 - val_loss: 0.3854 - val_accuracy: 0.8435 - 575ms/epoch - 6ms/step
Epoch 7/100
100/100 - 1s - loss: 0.3776 - accuracy: 0.8288 - val_loss: 0.3749 - val_accuracy: 0.8581 - 571ms/epoch - 6ms/step
Epoch 8/100
100/100 - 0s - loss: 0.3672 - accuracy: 0.8385 - val_loss: 0.3635 - val_accuracy: 0.8516 - 338ms/epoch - 3m

In [10]:
# test the model

test_loss, test_accuracy = model.evaluate(scaled_X_test, y_test)

20/20 [==============================] - 0s 2ms/step - loss: 0.0795 - accuracy: 0.9710


In [11]:
# Print Pretty

print('\nTest loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))


Test loss: 0.08. Test accuracy: 97.10%


In [36]:
# Save model
# Test accuracy for the saved model is 97%
tf.keras.models.save_model(model, '/content/drive/MyDrive/Colab Data/KY EPSB/model', include_optimizer=True)

## Predict Classification using model

In [13]:
# Grab the data that we want to predict
inputs = predict_df.loc[:, ['End Year Code', 'District Code', 'Level Code',
                            'Reported Spending per student', 'Money Difference per school', 'Proficiency Score', ]]

# Convert to np array and scale data
inputs = epsb_scaler.transform(inputs.values)

In [14]:
# Predict classification and convert to one column
pred_classification = model.predict(inputs).argmax(axis=1)

# Update the predict_df with the missing values in the Classification columns
predict_df['Classification Code'] = predict_df['Classification Code'].fillna(pd.Series(pred_classification))
predict_df['Classification'] = predict_df['Classification'].fillna(pd.Series(pred_classification)).replace([0, 1, 2], ['Needs Improvement', 'Proficient', 'Distinguished'])


398/398 [==============================] - 1s 1ms/step


# Predict Rating: a NN Model

## Shuffle, Split, Scale

In [15]:
nested_df = predict_df.loc[:,['End Year Code', 'Level Code',
                              'Reported Spending per student', 'Money Difference per school',
                              'Proficiency Score', 'Classification Code', 'Rating Code']]

nested_df.dropna(inplace=True)

#Shuffle data
nested_df = nested_df.sample(frac=1, random_state=7)
nested_df.reset_index(drop=True, inplace=True)

In [16]:
nested_inputs = nested_df.values[:,:-1]
nested_targets = nested_df.values[:,-1]

In [17]:
x_train, x_set, y_train, y_set = train_test_split(nested_inputs, nested_targets,
                                                  test_size=0.2, random_state=3, stratify = nested_targets)

x_valid, x_test, y_valid, y_test = train_test_split(x_set, y_set,
                                                    test_size=0.5, random_state=16, stratify = y_set)

In [18]:
# Scale data
nested_model_scaler = StandardScaler()

# Calculate and store the mean and sd
nested_model_scaler.fit(x_train)

# Apply the scaler
x_train_scaled = nested_model_scaler.transform(x_train)
x_valid_scaled = nested_model_scaler.transform(x_valid)
x_test_scaled = nested_model_scaler.transform(x_test)

In [19]:
nested_df = predict_df.loc[:,['End Year Code', #'District Code',
                              'Level Code',
                              'Reported Spending per student', 'Money Difference per school',
                              'Proficiency Score', 'Classification Code', 'Rating Code']]

nested_df.dropna(inplace=True)

#Shuffle data
nested_df = nested_df.sample(frac=1, random_state=7)
nested_df.reset_index(drop=True, inplace=True)

## Model

In [20]:
# Create the model
# Ran different rounds of the below model.  Some of the options I put in comments to the right.

input_size = 6
output_size = 5
hidden_layer_size = 10 # also tried 20, 5, 15, 30

nested_model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layer_size, activation='PReLU'), # relu, elu, PReLU, LeakyReLU, swish     runnerup: swish, PReLU
    tf.keras.layers.Dense(hidden_layer_size, activation='elu'), # relu, elu, swish, PReLU, gelu, tanh      runnerup: elu, PReLU
    tf.keras.layers.Dense(output_size, activation='softmax')
    ])

# Create a custom learning rate
custom_learning_rate = 0.005 # 0.01, 0.005, 0.001

# Compile the model
nested_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=custom_learning_rate),
                     loss='sparse_categorical_crossentropy',
                     metrics=['accuracy'])

# Fit the model
batch_size = 20  # 20, 50, 100, 30
max_epochs = 100

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

nested_model.fit(x_train_scaled, y_train,
          batch_size=batch_size,
          epochs = max_epochs,
          callbacks = [early_stopping],
          validation_data = (x_valid_scaled, y_valid),
          verbose = 2)


Epoch 1/100
101/101 - 1s - loss: 1.1734 - accuracy: 0.4965 - val_loss: 0.8500 - val_accuracy: 0.6349 - 1s/epoch - 13ms/step
Epoch 2/100
101/101 - 0s - loss: 0.6751 - accuracy: 0.7279 - val_loss: 0.5691 - val_accuracy: 0.7460 - 267ms/epoch - 3ms/step
Epoch 3/100
101/101 - 0s - loss: 0.4957 - accuracy: 0.7890 - val_loss: 0.4502 - val_accuracy: 0.7937 - 224ms/epoch - 2ms/step
Epoch 4/100
101/101 - 0s - loss: 0.4163 - accuracy: 0.8262 - val_loss: 0.4054 - val_accuracy: 0.8373 - 342ms/epoch - 3ms/step
Epoch 5/100
101/101 - 0s - loss: 0.3619 - accuracy: 0.8451 - val_loss: 0.3464 - val_accuracy: 0.8571 - 333ms/epoch - 3ms/step
Epoch 6/100
101/101 - 0s - loss: 0.3199 - accuracy: 0.8764 - val_loss: 0.2874 - val_accuracy: 0.8889 - 343ms/epoch - 3ms/step
Epoch 7/100
101/101 - 0s - loss: 0.2857 - accuracy: 0.8828 - val_loss: 0.2856 - val_accuracy: 0.8889 - 328ms/epoch - 3ms/step
Epoch 8/100
101/101 - 0s - loss: 0.2519 - accuracy: 0.9027 - val_loss: 0.2217 - val_accuracy: 0.9325 - 323ms/epoch - 3ms

In [21]:
# test the model
nested_test_loss, nested_test_accuracy = nested_model.evaluate(x_test_scaled, y_test)

# Print Pretty
print('\nNested Test loss: {0:.2f}. Nested Test accuracy: {1:.2f}%'.format(nested_test_loss, nested_test_accuracy*100.))

8/8 [==============================] - 0s 2ms/step - loss: 0.1159 - accuracy: 0.9603

Nested Test loss: 0.12. Nested Test accuracy: 96.03%


In [35]:
# Save model
# Test accuracy for the saved model is 96%
tf.keras.models.save_model(nested_model, '/content/drive/MyDrive/Colab Data/KY EPSB/nested_model', include_optimizer=True)

## Nested Predict

In [23]:
nested_inputs = predict_df.loc[:,['End Year Code', 'Level Code',
                              'Reported Spending per student', 'Money Difference per school',
                              'Proficiency Score', 'Classification Code', ]]

# Apply the scaler
nested_inputs = nested_model_scaler.transform(nested_inputs.values)

In [24]:
# Predict Ky Rating and convert to one column
predict_rating = nested_model.predict(nested_inputs).argmax(axis=1)

# Update the predict_df with the missing values in the Classification columns
predict_df['Rating Code'] = predict_df['Rating Code'].fillna(pd.Series(predict_rating))
predict_df['Rating'] = predict_df['Rating'].fillna(pd.Series(predict_rating)).replace([0,1,2,3,4], ['Very Low', 'Low', 'Medium', 'High', 'Very High'])


398/398 [==============================] - 1s 2ms/step


# Export and Impact View

In [25]:
predict_df

,End Year,End Year Code,District,District Code,School,School Code,Level,Level Code,Proficiency Score,Classification,Classification Code,Rating,Rating Code,Reported Spending per student,Money Difference per school
0,2012,0,Adair County,1,Adair County High School,1010,HS,2,55,Needs Improvement,0.00,Low,1.00,"6,460.00","-36,400.00"
1,2012,0,Adair County,1,John Adair Intermediate School,1013,ES,0,69,Proficient,1.00,Low,1.00,"7,200.00","-19,900.00"
2,2012,0,Adair County,1,Adair County Middle School,1014,MS,1,48,Needs Improvement,0.00,Very Low,0.00,"6,720.00","-55,700.00"
3,2012,0,Adair County,1,Adair County Elementary School,1016,ES,0,63,Proficient,1.00,Low,1.00,"7,260.00","-33,800.00"
4,2012,0,Allen County,5,Allen County Primary Center,5010,ES,0,50,Needs Improvement,0.00,Very Low,0.00,"7,930.00","190,500.00"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12718,2023,11,Woodford County,601,Simmons Elementary School,601075,ES,0,60,Needs Improvement,0.00,Medium,2.00,NaN,NaN
12719,2023,11,Woodford County,601,Woodford County High School,601084,HS,2,62,Needs Improvement,0.00,Medium,2.00,NaN,NaN
12720,2023,11,Woodford County,601,Woodford County Middle School,601085,MS,1,64,Needs Improvement,0.00,Medium,2.00,NaN,NaN
12721,2023,11,Woodford County,601,Huntertown Elementary School,601090,ES,0,70,Needs Improvement,0.00,Medium,2.00,NaN,NaN


In [26]:
predict_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12723 entries, 0 to 12722
Data columns (total 15 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   End Year                       12723 non-null  int64  
 1   End Year Code                  12723 non-null  int64  
 2   District                       12723 non-null  object 
 3   District Code                  12723 non-null  int64  
 4   School                         12723 non-null  object 
 5   School Code                    12723 non-null  int64  
 6   Level                          12723 non-null  object 
 7   Level Code                     12723 non-null  int64  
 8   Proficiency Score              12723 non-null  int64  
 9   Classification                 12723 non-null  object 
 10  Classification Code            12723 non-null  float64
 11  Rating                         12723 non-null  object 
 12  Rating Code                    12723 non-null 

In [27]:
predict_df.nunique()

End Year                           10
End Year Code                      10
District                          174
District Code                     174
School                           1223
School Code                      1249
Level                               3
Level Code                          3
Proficiency Score                 101
Classification                      3
Classification Code                 3
Rating                              5
Rating Code                         5
Reported Spending per student    1665
Money Difference per school      3491
dtype: int64

In [28]:
predict_df.to_csv('/content/drive/MyDrive/Colab Data/KY EPSB/All Years/predict_df.csv', index = False)

In [29]:
predict_df.describe()

,End Year,End Year Code,District Code,School Code,Level Code,Proficiency Score,Classification Code,Rating Code,Reported Spending per student,Money Difference per school
count,"12,723.00","12,723.00","12,723.00","12,723.00","12,723.00","12,723.00","12,723.00","12,723.00","11,288.00","11,386.00"
mean,"2,016.87",4.87,287.92,"288,040.45",0.61,65.13,0.66,1.25,"11,271.71","21,504.24"
std,3.48,3.48,163.92,"163,945.55",0.77,12.91,0.84,1.08,"4,313.99","117,681.25"
min,"2,012.00",0.00,1.00,"1,010.00",0.00,13.00,0.00,0.00,"2,340.00","-666,200.00"
25%,"2,014.00",2.00,165.00,"165,012.00",0.00,57.00,0.00,0.00,"8,200.00","-19,500.00"
50%,"2,016.00",4.00,275.00,"275,119.00",0.00,65.00,0.00,1.00,"10,280.00","22,800.00"
75%,"2,019.00",7.00,435.00,"435,040.00",1.00,74.00,1.00,2.00,"13,290.00","58,900.00"
max,"2,023.00",11.00,601.00,"601,120.00",2.00,123.00,2.00,4.00,"69,010.00","1,403,000.00"


In [33]:
def major_impacts(df, percentile):
    # Making sure the upper and lower is higher and lower, respectively
    upper, lower = (percentile, round(1 - percentile, 2)) if percentile >= (1 - percentile) else (round(1 - percentile, 2), percentile)
    cols = ['Money Difference per school', 'Proficiency Score']

    # For each column per year, get the literal values of the upper / lower by using the percents
    quantiles = df.groupby('End Year')[cols].apply(lambda x: x.quantile([upper, lower]))

    impacted_schools = pd.DataFrame()

    # Selects the rows with the corresponding year, and the literal high/ low values
    for year in df['End Year'].unique():
        year_df = df[df['End Year'] == year]
        high = quantiles.loc[(year, upper)]
        low = quantiles.loc[(year, lower)]

        # Uses the high / low values to select the rows that are on the outside of those values
        # Does this for each column x.name is the column name.  The output is a T/F column
        not_between_mask = ~year_df[cols].apply(lambda x: x.between(low[x.name], high[x.name], inclusive='neither'))

        # Takes the T/F column and selects the row for any row that has T.
        selected_rows = not_between_mask.any(axis=1)
        impacted_schools = pd.concat([impacted_schools, year_df[selected_rows]])

    return impacted_schools.reset_index(drop=True)


In [34]:
major_impacts(predict_df, 0.05)

,End Year,End Year Code,District,District Code,School,School Code,Level,Level Code,Proficiency Score,Classification,Classification Code,Rating,Rating Code,Reported Spending per student,Money Difference per school
0,2012,0,Allen County,5,Allen County Primary Center,5010,ES,0,50,Needs Improvement,0.00,Very Low,0.00,"7,930.00","190,500.00"
1,2012,0,Allen County,5,Allen County-Scottsville High School,5020,HS,2,51,Needs Improvement,0.00,Low,1.00,"8,500.00","157,000.00"
2,2012,0,Anchorage Independent,6,Anchorage Independent Public School,6010,MS,1,92,Distinguished,2.00,Low,1.00,"19,850.00","50,100.00"
3,2012,0,Ashland Independent,12,Hager Elementary School,12080,ES,0,76,Distinguished,2.00,Low,1.00,"6,450.00","31,900.00"
4,2012,0,Barren County,21,North Jackson Elementary,21014,ES,0,72,Distinguished,2.00,Low,1.00,"7,390.00","31,400.00"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3427,2023,11,Woodford County,601,Simmons Elementary School,601075,ES,0,60,Needs Improvement,0.00,Medium,2.00,NaN,NaN
3428,2023,11,Woodford County,601,Woodford County High School,601084,HS,2,62,Needs Improvement,0.00,Medium,2.00,NaN,NaN
3429,2023,11,Woodford County,601,Woodford County Middle School,601085,MS,1,64,Needs Improvement,0.00,Medium,2.00,NaN,NaN
3430,2023,11,Woodford County,601,Huntertown Elementary School,601090,ES,0,70,Needs Improvement,0.00,Medium,2.00,NaN,NaN
